# PSYCH 239: Week 2: Fully Connected Neural Network for Visual Classification


In [0]:
import torch
from torchvision import datasets, transforms


In [2]:
train_set = datasets.FashionMNIST('./data',
                           train=True, download=True,
                           transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307,), (0.3081,))
                             ]),
                           target_transform = None,)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw
Processing...
Done!


In [0]:
test_set = datasets.FashionMNIST('./data',
                           train=False, download=True,
                           transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307,), (0.3081,))
                             ]),
                           target_transform = None,)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=True)

In [0]:
class NIN(torch.nn.Module):
    def __init__(self):
        super(NIN, self).__init__()

        self.nin1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, 5, padding=2),
            torch.nn.ReLU(),
            torch.nn.Conv2d(32, 32, 1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(32, 32, 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(3, stride=2),
            torch.nn.Dropout(.5)          
        )
        self.nin2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64, 64, 1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64, 64, 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(3, stride=2),
            torch.nn.Dropout(.5)          
        )
        self.nin3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(128, 128, 1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(128, 10, 1),
            torch.nn.ReLU(),
        )
        self.pool = torch.nn.AvgPool2d(6, stride=1)  


    def forward(self, x):
        x = self.nin1(x)
        x = self.nin2(x)
        x = self.nin3(x)
        x = self.pool(x)
        return x.view(-1,10)

net = NIN().cuda()

In [0]:
mse_loss = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(net.parameters(), lr=1e-3)

In [0]:
def train_step(x, t, net, opt_fn, loss_fn):
    y = net(x.cuda())
    loss = loss_fn(y, t.cuda())
    loss.backward()
    opt_fn.step()
    opt_fn.zero_grad()
    return loss

In [12]:
x,t = next(iter(train_loader))
train_step(x, t, net, opt, mse_loss)

tensor(2.3039, device='cuda:0', grad_fn=<NllLossBackward>)

Train the network over multiple epochs

In [14]:
acc_hist_train = []
acc_hist_test = []
for epoch in range(5):
    acc_batch = []
    net.train()
    for x,t in iter(train_loader):   
        loss_train = train_step(x, t, net, opt, mse_loss)
        y = net(x.cuda()).cpu()
        acc_batch.append(torch.mean((t == y.argmax(1)).float()))
    acc_hist_train.append(torch.mean(torch.FloatTensor(acc_batch)))   
    print(loss_train)

    net.eval()
    acc_batch = []
    for x,t in iter(test_loader):   
        y = net(x.cuda()).cpu()
        acc_batch.append(torch.mean((t == y.argmax(1)).float()))
    acc_hist_test.append(torch.mean(torch.FloatTensor(acc_batch)))   

tensor(0.6348, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.8456, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6572, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.7046, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6050, device='cuda:0', grad_fn=<NllLossBackward>)


In [0]:
acc_hist_train

In [0]:
import pylab as plt
plt.plot(range(len(acc_hist_train)), acc_hist_train)
plt.plot(range(len(acc_hist_test)), acc_hist_test)